# Testing to get all liked songs of a user

In [204]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth
from datetime import datetime
import jwt

In [2]:
#sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials('bf1ba68423404778a60bcf3dee58d199','7365dc611a2d4ddba4ad61343f0b64d7'))

sp = spotipy.Spotify(auth_manager=SpotifyOAuth('86677c795a49463d9281fac012a87155','fe6f941da771447c920e02bbb2a82859', redirect_uri='http://localhost:5000',scope='user-library-read') )


In [3]:

user = sp.user('nobodyknowsmynam')
print(user)

{'display_name': 'Matt Laraia', 'external_urls': {'spotify': 'https://open.spotify.com/user/nobodyknowsmynam'}, 'followers': {'href': None, 'total': 9}, 'href': 'https://api.spotify.com/v1/users/nobodyknowsmynam', 'id': 'nobodyknowsmynam', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab6775700000ee854f8dbba7048b2fbd494b14df', 'width': None}], 'type': 'user', 'uri': 'spotify:user:nobodyknowsmynam'}


In [4]:
myPlaylists = sp.user_playlists('nobodyknowsmynam')

In [5]:
len(myPlaylists)

7

In [9]:
type(myPlaylists['items'])

list

In [10]:
for item in myPlaylists['items']:
    print(item['name'])

I guess so
raaage
Relax
Summer
Meditate
Positivity 😃
YES
H I T S
Jazz Vibes
Hype
Deep Focus
Songs to Sing in the Car


# Results:

Using a token, redirect URI, and scope, we can get the user's liked songs and maybe even extra songs they didn't like but put in their playlists.

In [11]:
sp.current_user_saved_tracks()

{'href': 'https://api.spotify.com/v1/me/tracks?offset=0&limit=20',
 'items': [{'added_at': '2023-03-24T16:11:54Z',
   'track': {'album': {'album_group': 'single',
     'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1grN0519h2zYqpRtYbDZAl'},
       'href': 'https://api.spotify.com/v1/artists/1grN0519h2zYqpRtYbDZAl',
       'id': '1grN0519h2zYqpRtYbDZAl',
       'name': 'Larry June',
       'type': 'artist',
       'uri': 'spotify:artist:1grN0519h2zYqpRtYbDZAl'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/0eVyjRhzZKke2KFYTcDkeu'},
       'href': 'https://api.spotify.com/v1/artists/0eVyjRhzZKke2KFYTcDkeu',
       'id': '0eVyjRhzZKke2KFYTcDkeu',
       'name': 'The Alchemist',
       'type': 'artist',
       'uri': 'spotify:artist:0eVyjRhzZKke2KFYTcDkeu'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/0c173mlxpT3dSFRgMO8XPh'},
       'href': 'https://api.spotify.com/v1/artists/0c173mlx

# Batch Processing of User's Liked Songs

In [23]:
track1=sp.current_user_saved_tracks(limit=30)['items'][1]['track']['id']

In [25]:
track2=sp.current_user_saved_tracks(limit=30)['items'][2]['track']['id']

In [75]:
def getRelevantTrackFeatures(sp,user_prior_login_date=''):
    track_ids = retrieveTrackIds(sp, user_prior_login_date)
    #track_ids = checkIdsInDatabase(track_ids)
    features = retrieveTrackFeatures(sp, track_ids)
    saveTrackFeaturesToDatabase(features)
    updateLastLoginDate(sp)
    return features

def retrieveTrackIds(sp, user_prior_login_date):
    track_ids = []
    offset = 0
    limit = 50
    liked_tracks = sp.current_user_saved_tracks(limit=limit, offset=offset)
    endLoopEarly = False

    while True:
        for item in liked_tracks['items']:
            if user_prior_login_date != '':
                if item['added_at']> user_prior_login_date:
                    track_ids.append(item['track']['id'])
                else:
                    endLoopEarly = True
                    break
            else:
                track_ids.append(item['track']['id'])
        offset += limit
        
        if len(liked_tracks['items']) < limit or endLoopEarly:
            # All tracks have been retrieved
            break
        
        liked_tracks = sp.current_user_saved_tracks(limit=limit, offset=offset)

    return track_ids

def checkIdsInDatabase(track_ids):
    #check if the track_ids are already in the database
    #if they are, remove them from the track_ids list
    for track_id in track_ids:
        if track_id in database:
            track_ids.remove(track_id)
    return track_ids



def retrieveTrackFeatures(sp, track_ids):
    features = []
    for i in range(0, len(track_ids), 50):
        features += sp.audio_features(track_ids[i:i+50])
        #only get the features we need, so we do not need type, uri, analysis_url, track_href
        for feature in features:
            del feature['type']
            del feature['uri']
            del feature['analysis_url']
            del feature['track_href']
    return features

def saveTrackFeaturesToDatabase(features):
    #save the features to the database
    for feature in features:
        database[feature['id']] = feature
    return database

def updateLastLoginDate(sp):
    #update the last login date in the database
    database['last_login_date'] = datetime.now()
    return 

In [58]:
track_features = retrieveTrackFeatures(sp, track_ids[:51])

In [60]:
track_features[0]

{'danceability': 0.637,
 'energy': 0.58,
 'key': 2,
 'loudness': -7.862,
 'mode': 1,
 'speechiness': 0.0512,
 'acousticness': 0.0951,
 'instrumentalness': 0,
 'liveness': 0.11,
 'valence': 0.514,
 'tempo': 78.509,
 'type': 'audio_features',
 'id': '2YALU0elzWXiee1nhHSWR6',
 'uri': 'spotify:track:2YALU0elzWXiee1nhHSWR6',
 'track_href': 'https://api.spotify.com/v1/tracks/2YALU0elzWXiee1nhHSWR6',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2YALU0elzWXiee1nhHSWR6',
 'duration_ms': 220127,
 'time_signature': 4}

In [51]:
print (len(track_ids))

3719


In [44]:
track_ids[0]

'7ijXfR09g8dImpTI586m7k'

In [57]:
len(track_ids)

3719

In [46]:
#get the audio features of track1
len(sp.audio_features(track_ids))

50

In [64]:
test_sample = sp.current_user_saved_tracks(limit=5, offset=50)

In [66]:
test_sample['items'][0]['added_at']

'2023-02-05T16:21:55Z'

In [69]:

date_str = test_sample['items'][0]['added_at']
dt = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%SZ')
dt

datetime.datetime(2023, 2, 5, 16, 21, 55)

In [70]:
most_recent = sp.current_user_saved_tracks(limit=1)

In [72]:
recent_dt_str=most_recent['items'][0]['added_at']
recent_dt = datetime.strptime(recent_dt_str, '%Y-%m-%dT%H:%M:%SZ')
recent_dt

datetime.datetime(2023, 3, 24, 16, 11, 54)

In [73]:
recent_dt>dt

True

In [80]:
import pandas as pd


In [81]:
track_featuresDF = pd.DataFrame(track_features)


In [84]:
track_featuresDF.shape

(51, 18)

# Spotipy Testing with integration

In [85]:
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd


#need to have credentials to access API
sp = spotipy.Spotify(auth_manager=SpotifyOAuth('86677c795a49463d9281fac012a87155','fe6f941da771447c920e02bbb2a82859', redirect_uri='http://localhost:5000',scope='user-library-read') )



In [231]:
from sklearn.preprocessing import StandardScaler

def getTrackMoodsIntoDB(sp,model, user_prior_login_date=''):
    features,track_ids = getRelevantTrackFeaturesOfUnlabelledLikedSongs(sp,user_prior_login_date=user_prior_login_date)
    preprocessedFeatures = clipAndNormalize(features)
    prediction, probablity = getMoodLabelMLP(model,preprocessedFeatures)
    #either this can return these 3 pieces of information or we can save them to the database and return nothing
    return prediction, probablity, track_ids



def getRelevantTrackFeaturesOfUnlabelledLikedSongs(sp,user_prior_login_date=''):
    track_ids = retrieveTrackIds(sp, user_prior_login_date)
    #track_ids = checkIdsInDatabase(track_ids)
    features = retrieveTrackFeatures(sp, track_ids)
    #saveTrackFeaturesToDatabase(features)
    #updateLastLoginDate(sp)
    return features, track_ids

def retrieveTrackIds(sp, user_prior_login_date):
    track_ids = []
    offset = 0
    limit = 50
    liked_tracks = sp.current_user_saved_tracks(limit=limit, offset=offset)
    endLoopEarly = False
    while True:
        for item in liked_tracks['items']:
            if user_prior_login_date != '':
                if item['added_at']> user_prior_login_date:
                    track_ids.append(item['track']['id'])
                else:
                    endLoopEarly = True
                    break
            else:
                track_ids.append(item['track']['id'])
        offset += limit
        
        if len(liked_tracks['items']) < limit or endLoopEarly:
            # All tracks have been retrieved
            break
        
        liked_tracks = sp.current_user_saved_tracks(limit=limit, offset=offset)

    return track_ids

def checkIdsInDatabase(track_ids):
    #check if the track_ids are already in the database
    #if they are, remove them from the track_ids list
    for track_id in track_ids:
        if track_id in database:
            track_ids.remove(track_id)
    return track_ids


def retrieveTrackFeatures(sp, track_ids):
    dfs = []
    for i in range(0, len(track_ids), 50):
        # Retrieve track features with current offset
        current_features = sp.audio_features(track_ids[i:i+50])
        
        # Convert to DataFrame
        df = pd.DataFrame(current_features)
        
        # Remove columns that we don't need
        df = df.drop(['type', 'uri', 'analysis_url', 'track_href','id'], axis=1)
        
        
        # Append to list of dataframes
        dfs.append(df)
    
    # Concatenate all dataframes into a single one
    features_df = pd.concat(dfs, ignore_index=True)
    
    features = features_df.to_numpy()
    
    return features_df


def saveTrackFeaturesToDatabase(features):
    #save the features to the database
    for feature in features:
        database[feature['id']] = feature
    return database

def updateLastLoginDate(sp):
    #update the last login date in the database
    database['last_login_date'] = datetime.now()
    return 



def clipAndNormalize(features):
    #clip the features to the range of the training data

    features['danceability'] = features['danceability'].clip(lower=0.22718080000000002, upper=0.906)
    features['energy'] = features['energy'].clip(lower=0.03545904, upper=0.978)
    features['loudness'] = features['loudness'].clip(lower=-26.4981552, upper=-1.6015904000000007)
    features['speechiness'] = features['speechiness'].clip(lower=0.0257, upper=0.46640959999999926)
    features['acousticness'] = features['acousticness'].clip(lower=8.353136000000001e-05, upper=0.9884095999999992)
    features['instrumentalness'] = features['instrumentalness'].clip(lower=0.0, upper=0.956)
    features['liveness'] = features['liveness'].clip(lower=0.0494, upper=0.697)
    features['valence'] = features['valence'].clip(lower=0.0382, upper=0.923)
    features['tempo'] = features['tempo'].clip(lower=63.7631808, upper=188.00344319999996)
    features['duration_ms'] = features['duration_ms'].clip(lower=88264.8768, upper=372339.1727999991)
    features['time_signature'] = features['time_signature'].clip(lower=3.0, upper=5.0)
    
    #normalize the data
    scaler = StandardScaler()
    preprocessedFeatures = scaler.fit_transform(features)
    return preprocessedFeatures



def getMoodLabelMLP(model,songFeautures):
        prediction = model.predict(songFeautures)
        pred_probability=model.predict_proba(songFeautures)
        return prediction, pred_probability

## experimenting with using requests to get all liked songs as opposed to spotipy

In [240]:
import base64
import requests
client_id = '86677c795a49463d9281fac012a87155'
client_secret = 'fe6f941da771447c920e02bbb2a82859'

# Encode client ID and secret as base64
client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode())

# Construct request headers
headers = {
    'Authorization': f'Basic {client_creds_b64.decode()}'
}

# Make request to Spotify API to obtain token
token_url = 'https://accounts.spotify.com/api/token'
data = {
    'grant_type': 'client_credentials',
    'scope': 'user-library-read'
}
response = requests.post(token_url, headers=headers, data=data)

# Extract access token from response
if response.status_code == 200:
    access_token = response.json()['access_token']
    print(f"Access token: {access_token}")
else:
    print(f"Error: {response.status_code}")



Access token: BQCN-AxORQeurDNxIxi3Y01Fdx5wi6kY02dfobQJjWz87VmQVx44O1hXoOEZR-I5t2KEORkBy9KQEbmpG4h0w4Qxm79dEBzfXg5QXSmIZJGDSJ8EsftvjHYA-fk4g3PVEg


In [241]:
access_token

'BQCN-AxORQeurDNxIxi3Y01Fdx5wi6kY02dfobQJjWz87VmQVx44O1hXoOEZR-I5t2KEORkBy9KQEbmpG4h0w4Qxm79dEBzfXg5QXSmIZJGDSJ8EsftvjHYA-fk4g3PVEg'

In [242]:
import requests
def retrieveTrackFeaturesToken(access_token, track_ids):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    dfs = []
    for i in range(0, len(track_ids), 50):
        # Construct the URL for the current offset
        url = 'https://api.spotify.com/v1/audio-features'
        params = {'ids': ','.join(track_ids[i:i+50])}
        
        # Make the request to the API
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            # Convert the response to a DataFrame
            data = response.json()['audio_features']
            df = pd.DataFrame(data)
            # Remove columns that we don't need
            df = df.drop(['type', 'uri', 'analysis_url', 'track_href','id'], axis=1)
            
            # Append to list of dataframes
            dfs.append(df)
        else:
            print('Error: ', response.status_code)
    
    # Concatenate all dataframes into a single one
    features_df = pd.concat(dfs, ignore_index=True)
    
    return features_df

In [243]:
def retrieveTrackIdsToken(access_token, user_prior_login_date):
    print(access_token)
    track_ids = []
    offset = 0
    limit = 50
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    url = 'https://api.spotify.com/v1/me/tracks'
    params = {'limit': limit, 'offset': offset}
    endLoopEarly = False
    while True:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            liked_tracks = response.json()
            for item in liked_tracks['items']:
                if user_prior_login_date != '':
                    if item['added_at'] > user_prior_login_date:
                        track_ids.append(item['track']['id'])
                    else:
                        endLoopEarly = True
                        break
                else:
                    track_ids.append(item['track']['id'])
            offset += limit

            if len(liked_tracks['items']) < limit or endLoopEarly:
                # All tracks have been retrieved
                break
            params = {'limit': limit, 'offset': offset}
        else:
            print('Error: ', response.text)
            break
    return track_ids

In [244]:
track_ids = retrieveTrackIds(sp,'')

In [245]:
track_ids

['5aUJonYqwhmvgNlWQdH6P1',
 '2iu60eTIm2py6MaoVMcBAR',
 '6TMwCGsTOlt796wpmiFIt7',
 '3SsJ17EnPIu1B4GZshqjIS',
 '3fK1ALymCKkWjVPTFWFDsl',
 '1JKcdDCv8Yk39VRWMxx8Lj',
 '3rOZKd4FOb5fA32TSd1IiR',
 '3yKJI6wr8NR0ITCLEW3nvc',
 '5imShWWzwqfAJ9gXFpGAQh',
 '3XVBdLihbNbxUwZosxcGuJ',
 '285hMzLhJwHVLe9QT9qilk',
 '6MDTpYxPFL0wtFd0pjfKws',
 '6Qyc6fS4DsZjB2mRW9DsQs',
 '1HpAxf0kdqZSAisutLE5CW',
 '1z81jqjgmEGJo5kf19vVa8',
 '3ujWwlBBHka38mAH2y4Wi5',
 '767iOXvMCZC0W8HcPJm1am',
 '6clDsO8HwhHEgJDDp88VdL',
 '2kimLa9Di1xK5MpZSxJhRy',
 '4AUwKqmQeAplXNXX2yg10g',
 '3zIcvn5r3QGea6a4M36MQr',
 '0G8Wt2jPfzPJCx01EUtZSI',
 '6O2cLm5tVQbk5pbHdj1YlO',
 '3FAJ6O0NOHQV8Mc5Ri6ENp',
 '7oEFDtEebqQH4yMfSA1MOM',
 '7BbaIYAdi3pg4MGl6PHwPv',
 '2Qr5PWUt8XycAAv338LicY',
 '0jQ4mEnWB1AuSVqnFLhxT6',
 '5NqIHUcwfBDpc1rgTxKvyG',
 '4xJQDZxCjAJdemvWoBR9nL',
 '1MJ5f5EYBC92ADD6xcz7nb',
 '55Am8neGJkdj2ADaM3aw5H',
 '0AICBlLzFCTpUqmAbtzB2z',
 '0FA4wrjDJvJTTU8AepZTup',
 '7hrcHfHWaLhooHzGltlCYB',
 '1jcGbrSCZ32Q2vyvOpOZzg',
 '7stPYRMF2APq5Jzf38vLST',
 

In [246]:
featuresToken = retrieveTrackFeaturesToken(access_token, track_ids[:50])
featuresToken.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.462,0.809,5,-6.668,1,0.3260,0.21400,0.000000,0.470,0.598,76.826,129752,4
1,0.622,0.501,0,-7.325,1,0.0816,0.78300,0.000004,0.301,0.512,81.478,159033,4
2,0.726,0.746,5,-8.740,0,0.2830,0.08870,0.000558,0.382,0.692,129.990,179697,4
3,0.491,0.770,5,-6.018,0,0.3080,0.11200,0.000000,0.319,0.370,177.977,189427,3
4,0.576,0.716,1,-6.626,1,0.2280,0.00978,0.000000,0.135,0.458,80.012,171000,4


In [229]:
features = retrieveTrackFeatures(sp, track_ids[0:50])
features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.462,0.809,5,-6.668,1,0.3260,0.21400,0.000000,0.470,0.598,76.826,129752,4
1,0.622,0.501,0,-7.325,1,0.0816,0.78300,0.000004,0.301,0.512,81.478,159033,4
2,0.726,0.746,5,-8.740,0,0.2830,0.08870,0.000558,0.382,0.692,129.990,179697,4
3,0.491,0.770,5,-6.018,0,0.3080,0.11200,0.000000,0.319,0.370,177.977,189427,3
4,0.576,0.716,1,-6.626,1,0.2280,0.00978,0.000000,0.135,0.458,80.012,171000,4


In [247]:
track_ids_from_token = retrieveTrackIdsToken(access_token, '')

BQCN-AxORQeurDNxIxi3Y01Fdx5wi6kY02dfobQJjWz87VmQVx44O1hXoOEZR-I5t2KEORkBy9KQEbmpG4h0w4Qxm79dEBzfXg5QXSmIZJGDSJ8EsftvjHYA-fk4g3PVEg
Error:  {
  "error" : {
    "status" : 401,
    "message" : "Missing token"
  }
}


## A continuation of using spotipy from earlier (I should have put the other code from the header above at the bottom of the notebook)

In [165]:
features = retrieveTrackFeatures(sp, track_ids[0:50])

In [166]:
features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.726,0.746,5,-8.740,0,0.2830,0.08870,0.000558,0.3820,0.692,129.990,179697,4
1,0.491,0.770,5,-6.018,0,0.3080,0.11200,0.000000,0.3190,0.370,177.977,189427,3
2,0.576,0.716,1,-6.626,1,0.2280,0.00978,0.000000,0.1350,0.458,80.012,171000,4
3,0.640,0.570,0,-6.021,1,0.0344,0.23500,0.000002,0.1130,0.233,141.923,233227,4
4,0.662,0.467,8,-10.151,1,0.2830,0.24000,0.000003,0.0993,0.392,169.235,115446,4


In [148]:
resultsForAllMyPersonalLabels = getTrackMoodsIntoDB(sp,loaded_model)

In [155]:
#find how many results were scored 0 through 7 in the numpy array of predictions
for i in range(0,8):
    print("Number of songs scored as ", i, " is ", np.count_nonzero(resultsForAllMyPersonalLabels[0] == i))

Number of songs scored as  0  is  64
Number of songs scored as  1  is  353
Number of songs scored as  2  is  334
Number of songs scored as  3  is  2396
Number of songs scored as  4  is  219
Number of songs scored as  5  is  79
Number of songs scored as  6  is  73
Number of songs scored as  7  is  203


In [157]:
resultsForAllMyPersonalLabels[2]

['spotify:track:5aUJonYqwhmvgNlWQdH6P1',
 'spotify:track:2iu60eTIm2py6MaoVMcBAR',
 'spotify:track:6TMwCGsTOlt796wpmiFIt7',
 'spotify:track:3SsJ17EnPIu1B4GZshqjIS',
 'spotify:track:3fK1ALymCKkWjVPTFWFDsl',
 'spotify:track:1JKcdDCv8Yk39VRWMxx8Lj',
 'spotify:track:3rOZKd4FOb5fA32TSd1IiR',
 'spotify:track:3yKJI6wr8NR0ITCLEW3nvc',
 'spotify:track:5imShWWzwqfAJ9gXFpGAQh',
 'spotify:track:3XVBdLihbNbxUwZosxcGuJ',
 'spotify:track:285hMzLhJwHVLe9QT9qilk',
 'spotify:track:6MDTpYxPFL0wtFd0pjfKws',
 'spotify:track:6Qyc6fS4DsZjB2mRW9DsQs',
 'spotify:track:1HpAxf0kdqZSAisutLE5CW',
 'spotify:track:1z81jqjgmEGJo5kf19vVa8',
 'spotify:track:3ujWwlBBHka38mAH2y4Wi5',
 'spotify:track:767iOXvMCZC0W8HcPJm1am',
 'spotify:track:6clDsO8HwhHEgJDDp88VdL',
 'spotify:track:2kimLa9Di1xK5MpZSxJhRy',
 'spotify:track:4AUwKqmQeAplXNXX2yg10g',
 'spotify:track:3zIcvn5r3QGea6a4M36MQr',
 'spotify:track:0G8Wt2jPfzPJCx01EUtZSI',
 'spotify:track:6O2cLm5tVQbk5pbHdj1YlO',
 'spotify:track:3FAJ6O0NOHQV8Mc5Ri6ENp',
 'spotify:track:

In [147]:
features[0]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.462,0.809,5,-6.668,1,0.3260,0.214000,0.000000,0.4700,0.598,76.826,129752,4
1,0.622,0.501,0,-7.325,1,0.0816,0.783000,0.000004,0.3010,0.512,81.478,159033,4
2,0.726,0.746,5,-8.740,0,0.2830,0.088700,0.000558,0.3820,0.692,129.990,179697,4
3,0.491,0.770,5,-6.018,0,0.3080,0.112000,0.000000,0.3190,0.370,177.977,189427,3
4,0.576,0.716,1,-6.626,1,0.2280,0.009780,0.000000,0.1350,0.458,80.012,171000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,0.736,0.604,1,-7.357,0,0.2220,0.065200,0.000064,0.1240,0.574,155.007,264532,4
3717,0.736,0.795,1,-3.299,0,0.0545,0.498000,0.000001,0.2540,0.511,104.066,230813,4
3718,0.748,0.546,8,-8.134,1,0.1060,0.590000,0.000000,0.1120,0.413,140.005,220880,4
3719,0.712,0.623,8,-5.518,1,0.0488,0.026800,0.022900,0.0825,0.473,96.766,235352,4


In [98]:
import os
os.chdir('C:/Users/mlar5/OneDrive/Desktop/Code Folder/198 Senior Design/Models/Spotipy-data-models')


In [138]:
features[0]

array([[ 1.08006127e+00, -3.31602400e-01, -1.11614187e+00,
         2.10949982e-01, -1.77951304e+00,  1.17041565e+00,
        -9.10037199e-01, -3.34417793e-01, -2.11960964e-01,
         1.29059462e+00,  1.06520569e+00, -1.88687070e+00,
         2.77350098e-01],
       [ 1.19020737e+00,  4.10560860e-01, -8.45233645e-01,
         2.84210886e-01, -1.77951304e+00,  2.72905926e+00,
        -9.46558284e-01, -3.34417793e-01, -6.66217793e-01,
         2.86428236e-01,  1.60388129e+00, -7.83832227e-01,
         2.77350098e-01],
       [ 1.33969138e+00,  2.41887392e-01, -1.11614187e+00,
        -2.92556934e-01, -1.77951304e+00,  1.32125213e+00,
         1.76981863e-01, -3.34417793e-01, -5.30771313e-01,
         4.95440243e-01,  1.95274852e-01, -1.17613585e+00,
         2.77350098e-01],
       [ 2.16578719e+00, -5.19552836e-01, -3.25089864e-02,
        -2.86861740e-01, -1.77951304e+00,  3.88547226e+00,
         3.04461119e-01, -3.34417793e-01, -6.71328981e-01,
        -2.36101780e-01,  4.34193397e

In [99]:
#load the model MLP1.pkl
import pickle
filename = 'MLP1.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [146]:
loaded_model.predict(features[0])

c:\Users\mlar5\anaconda3\envs\TextMining\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


array([3, 3, 3, ..., 3, 3, 3], dtype=int64)

In [140]:
features[1][47]

'spotify:track:1WWRakZzeLuWPlfymWPSYf'

In [111]:
features[1][0]

array([ 6.70000e-01,  5.17000e-01,  1.00000e+00, -6.70700e+00,
        0.00000e+00,  1.65000e-01,  2.25000e-02,  0.00000e+00,
        1.48000e-01,  6.71000e-01,  1.62013e+02,  9.62960e+04,
        4.00000e+00])

# Advanced song mood labelling integration testing

In [132]:
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from sklearn.discriminant_analysis import StandardScaler
import pickle
from datetime import datetime
from os import chdir
import requests
from bs4 import BeautifulSoup
from bs4 import element
from copy import deepcopy
import re

#need to have credentials to access API
sp = spotipy.Spotify(auth_manager=SpotifyOAuth('86677c795a49463d9281fac012a87155','fe6f941da771447c920e02bbb2a82859', redirect_uri='http://localhost:5000',scope='user-library-read') )



In [90]:
chdir('C:/Users/mlar5/OneDrive/Desktop/Code Folder/198 Senior Design/Models/Spotipy')

## verifying song id retrieval

In [38]:
def retrieveTrackIds(sp, user_prior_login_date):
    track_ids = []
    offset = 0
    limit = 50
    liked_tracks = sp.current_user_saved_tracks(limit=limit, offset=offset)
    endLoopEarly = False
    while True:
        for item in liked_tracks['items']:
            if user_prior_login_date != '':
                if datetime.strptime(item['added_at'],'%Y-%m-%dT%H:%M:%SZ')> user_prior_login_date:
                    track_ids.append(item['track']['id'])
                else:
                    endLoopEarly = True
                    break
            else:
                track_ids.append(item['track']['id'])
        offset += limit
        
        if len(liked_tracks['items']) < limit or endLoopEarly:
            # All tracks have been retrieved
            break
        
        liked_tracks = sp.current_user_saved_tracks(limit=limit, offset=offset)
    
    return track_ids

### playing around with datetime (skip)

In [2]:
# get the users liked songs
results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " - ", track['name'])
    break

0 Cochise  –  PLASMA


In [35]:
#convert results['items'][0]['added_at'] to datetime object

date = datetime.strptime(results['items'][0]['added_at'], '%Y-%m-%dT%H:%M:%SZ')

In [25]:
results['items'][0]['added_at']

'2023-04-03T20:33:05Z'

In [15]:
date

datetime.datetime(2023, 4, 3, 20, 33, 5)

In [36]:
now = datetime.utcnow()
if date > now:
    print("yes")
else:
    print("no")


#get the datetime of 5 hours ago

from datetime import datetime, timedelta
six_hours_ago = datetime.utcnow() - timedelta(hours=6)
if date > six_hours_ago:
    print("yes")
else:
    print("no")

no
yes


In [23]:
#get the current time with the datetime library in the same format as the date from the API
now = datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ')

#convert now to datetime object
now = datetime.strptime(now, '%Y-%m-%dT%H:%M:%SZ')

if date > now:
    print("yes")
else:
    print("no")

yes


### testing id retrieval based on a given date

within the last 6 hours i liked 2 songs, which is what i get as expected below

In [39]:
ids = retrieveTrackIds(sp, six_hours_ago)

In [40]:
ids

['5Vp7xBUxV22dCkvUaCOjES', '2MHzmX7IqQWemKQRomNgA3']

In [41]:
retrieveTrackIds(sp, now)

[]

## verifying conversion of features to dictionary with song id as key

In [59]:
def retrieveTrackFeatures(sp, track_ids):
    dfs = []
    for i in range(0, len(track_ids), 50):
        # Retrieve track features with current offset
        current_features = sp.audio_features(track_ids[i:i+50])
        
        # Convert to DataFrame
        df = pd.DataFrame(current_features)
        
        # Remove columns that we don't need
        df = df.drop(['type', 'uri', 'analysis_url', 'track_href'], axis=1)
        
        
        # Append to list of dataframes
        dfs.append(df)
    
    # Concatenate all dataframes into a single one
    features_df = pd.concat(dfs, ignore_index=True)
    
    
    #convert to dictionary, with track id as key
    #featuresDict = features_df.set_index('id').T.to_dict('list')
    
    return features_df


In [60]:
featuresDF = retrieveTrackFeatures(sp, ids)

In [61]:
featuresDF

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,0.695,0.489,10,-9.091,1,0.1460,0.03060,0.000000,0.417,0.622,141.028,5Vp7xBUxV22dCkvUaCOjES,123617,4
1,0.716,0.606,7,-10.203,1,0.0507,0.00337,0.000168,0.168,0.223,108.762,2MHzmX7IqQWemKQRomNgA3,175448,3


## verifying conversion of features to dataframe

In [91]:
def clipAndNormalize(features):
    #clip the features to the range of the training data
    features['danceability'] = features['danceability'].clip(lower=0.22718080000000002, upper=0.906)
    features['energy'] = features['energy'].clip(lower=0.03545904, upper=0.978)
    features['loudness'] = features['loudness'].clip(lower=-26.4981552, upper=-1.6015904000000007)
    features['speechiness'] = features['speechiness'].clip(lower=0.0257, upper=0.46640959999999926)
    features['acousticness'] = features['acousticness'].clip(lower=8.353136000000001e-05, upper=0.9884095999999992)
    features['instrumentalness'] = features['instrumentalness'].clip(lower=0.0, upper=0.956)
    features['liveness'] = features['liveness'].clip(lower=0.0494, upper=0.697)
    features['valence'] = features['valence'].clip(lower=0.0382, upper=0.923)
    features['tempo'] = features['tempo'].clip(lower=63.7631808, upper=188.00344319999996)
    features['duration_ms'] = features['duration_ms'].clip(lower=88264.8768, upper=372339.1727999991)
    features['time_signature'] = features['time_signature'].clip(lower=3.0, upper=5.0)
    
    #normalize the data
    scaler = pickle.load(open('scaler1.pkl', 'rb'))
    #fit on all columns except the track id
    rawfeatures = features.drop(['id'], axis=1)
    preprocessedFeatures = scaler.transform(rawfeatures)

    #convert to dictionary, with track id as key
    preprocessedFeatures = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)
    preprocessedFeatures['id']= features['id']
    preprocessedFeatures = preprocessedFeatures.set_index('id').T.to_dict('list')
    return preprocessedFeatures


In [94]:
clipAndNormalize(featuresDF)

{'5Vp7xBUxV22dCkvUaCOjES': [0.5633998198962562,
  -0.2288413748774762,
  1.33879168378533,
  -0.02412074364979456,
  0.8056064047770485,
  0.5241219654311546,
  -0.9664071527188705,
  -0.5569624156327477,
  2.023227611096207,
  1.044946273718787,
  0.6694072548842553,
  -1.2795042419450267,
  0.1873415855961101],
 '2MHzmX7IqQWemKQRomNgA3': [0.7053298848670134,
  0.22928570535959303,
  0.4967945227568651,
  -0.23197629117833288,
  0.8056064047770485,
  -0.45989012320219097,
  -1.0450418225826095,
  -0.556483988553163,
  -0.00524372044159579,
  -0.7221847710643696,
  -0.4073731689831935,
  -0.3458834190756754,
  -3.2980007131677196]}

In [83]:
featuresDF

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,0.695,0.489,10,-9.091,1,0.1460,0.03060,0.000000,0.417,0.622,141.028,5Vp7xBUxV22dCkvUaCOjES,123617,4
1,0.716,0.606,7,-10.203,1,0.0507,0.00337,0.000168,0.168,0.223,108.762,2MHzmX7IqQWemKQRomNgA3,175448,3


In [75]:
#remove the id column in featuresDF
featuresDF.drop(['id'], axis=1)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.695,0.489,10,-9.091,1,0.1460,0.03060,0.000000,0.417,0.622,141.028,123617,4
1,0.716,0.606,7,-10.203,1,0.0507,0.00337,0.000168,0.168,0.223,108.762,175448,3


## verify scraper inputs

In [145]:
def getTitlesAndArtists(sp, track_ids):
    titleArtistPairs = {}
    for i in range(0,len(track_ids),50):
        tracks = sp.tracks(track_ids[i:i+50])
        for track in tracks['tracks']:
            title=track['name']
            #check if the track ends with (feat. artist) using a regex
            if re.search(r' \(feat. .*\)$', title):
                #remove the (feat. artist) from the title
                title = re.sub(r' \(feat. .*\)$', '', title)

            artists=[]
            for artist in track['artists']:
                artists.append(artist['name'])
            titleArtistPairs[track['id']] = {'title':title,'artist(s)':artists}

    return titleArtistPairs

In [134]:
getTitlesAndArtists(sp,ids)

{'5Vp7xBUxV22dCkvUaCOjES': {'title': 'PLASMA', 'artist(s)': ['Cochise']},
 '2MHzmX7IqQWemKQRomNgA3': {'title': 'Stonecrest', 'artist(s)': ['Lil Gnar']}}

In [135]:

three_ids = deepcopy(ids)
three_ids.append('2c44NyHXDy1lIfuoSOufVi')

In [146]:
scraperInputs = getTitlesAndArtists(sp,three_ids)
scraperInputs 

{'5Vp7xBUxV22dCkvUaCOjES': {'title': 'PLASMA', 'artist(s)': ['Cochise']},
 '2MHzmX7IqQWemKQRomNgA3': {'title': 'Stonecrest', 'artist(s)': ['Lil Gnar']},
 '2c44NyHXDy1lIfuoSOufVi': {'title': 'Go Down',
  'artist(s)': ['Don Toliver', 'TisaKorean']}}

## Testing scraper outputs
Did not work with the song that its title contained (feat. XXXX) in it. I think it's because the song title is not in the same format as the other songs. I'll have to look into this to see if it also had something to do with the two artist names

Update: (feat. XXXX) is the problem

In [137]:
#Helps parse miscellaneous tags <i>, </br>, etc,.
def _lyricsHelper(html, lyrics_list):
    for tag in html.childGenerator():
        if type(tag) == element.NavigableString:
            _handleLyricAppend(lyrics_list, tag.text.strip())
        elif tag.name == 'br' and lyrics_list[len(lyrics_list) - 1] != '':
            lyrics_list.append('')
        elif html.name == 'a':
            _lyricsHelper(tag, lyrics_list)

#Reads the HTML for lyrics dividers (if they exist) and appends the lyrics line by line to a list
def _getLyricsFromHTML(html):
    lyrics = html.findAll("div", {"data-lyrics-container" : "true"})
    lyrics_list = ['']
    for segment in lyrics:
        for a in segment.childGenerator():
            lyric = None
            if type(a) == element.NavigableString:
                lyric = a.strip()
                _handleLyricAppend(lyrics_list, lyric)
            else:
                _lyricsHelper(a, lyrics_list)
            if a.name == 'br' and lyrics_list[len(lyrics_list) - 1] != '':
                lyrics_list.append('')
    return lyrics_list

#Helper function to handle appending and manipulating lyrics_list. A new line is generated only for </br> tags
def _handleLyricAppend(lyrics_list, lyric):
    if lyric is not None:
        last_index = len(lyrics_list) - 1
        #Handle special cases (parenthesis and symbols stick with words for instance)
        if lyrics_list[last_index] != '' and (lyrics_list[last_index][-1] in ['(','[','{','<'] or lyric in [')',']','}','>','!','?',',','.']):
            lyrics_list[last_index] += lyric
        else:
            lyrics_list[last_index] += " " + lyric if lyrics_list[last_index] != '' else lyric

#Determines whether a song will need to be translated (returns the link if it does, otherwise returns None)
def _getSongTranslationLink(html):
    translation_tags = html.findAll('a', class_='TextButton-sc-192nsqv-0 hVAZmF LyricsControls__TextButton-sghmdv-6 hXTHjZ')
    for tag in translation_tags:
        if "english-translations" in tag['href']:
            return tag['href']
    return None

#Determines whether a page exists
def _pageExists(html):
    return html.find('div', class_='render_404') == None
        
#function to scrape lyrics from genius, takes an array of artists, and songname
def scrapeLyrics(artistnames, songname):
    lyrics_list = []
    found = False
    i = 0
    html = None
    while i < len(artistnames) and not(found):
        artistname = artistnames[i]
        artistname2 = str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)
        songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
        page_url = 'https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics'
        page = requests.get(page_url)
        html = BeautifulSoup(page.text, 'html.parser') 
        found = _pageExists(html)
        i += 1
    if found:
        #check if there is an english translation
        translation_url = _getSongTranslationLink(html)
        if translation_url is not None:
            page = requests.get(translation_url)
            html = BeautifulSoup(page.text, 'html.parser') 
            lyrics_list = _getLyricsFromHTML(html)
        else:
            #If there isn't a translation, make sure it's in english in the first place
            english = False
            for script in html.findAll('script'):
                if "language\\\":\\\"en" in str(script):
                    english = True
            if english: lyrics_list = _getLyricsFromHTML(html)
    return lyrics_list
    

In [141]:
def getScrapedLyrics(scraperInputs):
        all_lyrics_dict = {}
        for id, songInfo in scraperInputs.items():
                #maybe add a sleep or something to prevent getting blocked
                lyrics = scrapeLyrics(songInfo['artist(s)'],songInfo['title'])
                if len(lyrics) > 0:
                        all_lyrics_dict[id]=lyrics
        return all_lyrics_dict

In [147]:
all_lyrics_dict= getScrapedLyrics(scraperInputs)

In [148]:
all_lyrics_dict.keys()

dict_keys(['5Vp7xBUxV22dCkvUaCOjES', '2MHzmX7IqQWemKQRomNgA3', '2c44NyHXDy1lIfuoSOufVi'])

In [129]:
# change scraperInputs 2c44NyHXDy1lIfuoSOufVi title to 'Go Down'
scraperInputs['2c44NyHXDy1lIfuoSOufVi']['title'] = 'Go Down'

In [130]:
all_lyrics_dict= getScrapedLyrics(scraperInputs)

In [131]:
all_lyrics_dict.keys()

dict_keys(['5Vp7xBUxV22dCkvUaCOjES', '2MHzmX7IqQWemKQRomNgA3', '2c44NyHXDy1lIfuoSOufVi'])